In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
sys.setrecursionlimit(5000)

In [3]:
from filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
    get_unique_nutzung,
)

from compute_ET_green import compute_et_green, calculate_band_std_dev

from utils.ee_utils import (
    back_to_float,
    back_to_int,
    export_image_to_asset,
    print_value_ranges,
)
from utils import date_utils, ee_utils

from typing import List, Tuple

import ee
import geemap

In [4]:
ee.Initialize(project="thurgau-irrigation")

## Compute and export ET Green


In [5]:
TIME_STEPS = 12
TIME_STEP_TYPE = "monthly"
YEARS = range(2022, 2023)

In [6]:
DOUBLE_CROPPING_IMAGE = ee.Image(
    "projects/thurgau-irrigation/assets/Zuerich/crop_vegetation_period_zh_2022"
)

In [7]:
# path_to_WaPOR_ET_asset = "projects/thurgau-irrigation/assets/Thurgau/ET_WaPOR_10m_dekadal_test_refactor"


# ET_COLLECTIONs = ee_utils.merge_collections(
#     years=YEARS, asset_name=path_to_WaPOR_ET_asset
# )

# ET_COLLECTION = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/Zuerich/ET_WaPOR_10m_dekadal_2022"
# ).map(lambda img: back_to_float(img, 100))

# ET_COLLECTION = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/ETlandsatmonthly"
# ).filterDate("2022-01-01", "2022-12-31")


# # Replace negative pixel values with 0 while retaining image properties
# def set_negative_to_zero(img):
#     return img.max(0).copyProperties(img, img.propertyNames())

# ET_COLLECTION = ET_COLLECTION.map(set_negative_to_zero)
# ET_COLLECTION = date_utils.set_to_first_of_month(ET_COLLECTION)

# ee_utils.print_value_ranges(ET_COLLECTION, "ET")
# date_utils.print_collection_dates(ET_COLLECTION)

ET_COLLECTION = ee.ImageCollection("projects/thurgau-irrigation/assets/Zuerich/Landsat_ET_monthly_downscaled_10m_ZH_2022").map(lambda img: back_to_float(img, 100))

date_utils.print_collection_dates(ET_COLLECTION)
ee_utils.print_value_ranges(ET_COLLECTION, "downscaled")

Dates of images in the collection:
2022-01-01
2022-02-01
2022-03-01
2022-04-01
2022-05-01
2022-06-01
2022-07-01
2022-08-01
2022-09-01
2022-10-01
2022-11-01
2022-12-01
Image 1: Min = -8849.86, Max = 44657.47
Image 2: Min = -182.27, Max = 230.55
Image 3: Min = -539.40, Max = 430.08
Image 4: Min = -228.12, Max = 336.65
Image 5: Min = -316.47, Max = 710.06
Image 6: Min = -2295.27, Max = 944.03
Image 7: Min = -2996.71, Max = 614.55
Image 8: Min = -1812.83, Max = 743.15
Image 9: Min = -327.99, Max = 518.09
Image 10: Min = -27.86, Max = 2990.37
Image 11: Min = 0.00, Max = 0.00
Image 12: Min = 0.00, Max = 0.00


In [8]:
ET_COLLECTION.first().bandNames().getInfo()

['downscaled']

In [9]:
PATH_TO_AOI = "projects/thurgau-irrigation/assets/Zuerich/Zuerich_bound"

aoi_feature_collection = ee.FeatureCollection(PATH_TO_AOI)
aoi_geometry = aoi_feature_collection.geometry().simplify(500)

AOI = aoi_geometry.buffer(100)

In [10]:
def process_and_export_et_green(
    et_collection_list: ee.List,
    year: int,
    aoi: ee.Geometry,
    time_steps: int = 36,
    time_step_type: str = "dekadal",
    double_cropping_image: ee.Image = None,
) -> List[ee.batch.Task]:
    """
    Process and export ET green images for a given year.

    Args:
        et_collection (ee.List): List of ET images
        year (int): Year to process
        aoi (ee.Geometry): Area of interest
        time_steps (int): Number of time steps (36 for dekadal, 12 for monthly)
        time_step_type (str): Type of time step ("dekadal" or "monthly")
        double_cropping_image (ee.Image): Double cropping image

    Returns:
        List[ee.batch.Task]: List of export tasks
    """
    nutzung_collection = ee.FeatureCollection(
        f"projects/thurgau-irrigation/assets/ZH_Nutzungsflaechen_2/ZH_Nutzungsflaechen_{year}"
    )

    # Uncomment this if not working with Zuerich!
    nutzung_collection = nutzung_collection.map(
        lambda feature: feature.set("nutzung", feature.get("NUTZUNG"))
    )

    jurisdictions = ee.FeatureCollection(
        f"projects/thurgau-irrigation/assets/Zuerich/zurich_jurisdictions_{year}_wgs84"
    )

    double_cropping = ee.Image(double_cropping_image)

    # Prepare crop filters
    not_irrigated_crops = get_crops_to_exclude()
    rainfed_crops = get_rainfed_reference_crops()
    exclude_filter, rainfed_filter = create_crop_filters(
        not_irrigated_crops, rainfed_crops
    )

    # Process nutzung collection
    nutzung_with_double_crop = add_double_cropping_info(
        nutzung_collection, double_cropping
    )
    _, rainfed_fields = filter_crops(
        nutzung_with_double_crop, exclude_filter, rainfed_filter
    )

    tasks = []
    for i in range(time_steps):
        # Get time step info
        if time_step_type == "dekadal":
            dekadal = i % 3 + 1
            month = i // 3 + 1
            time_step_name = f"{month:02d}_D{dekadal}"
        elif time_step_type == "monthly":
            month = i + 1
            time_step_name = f"{month:02d}"
        else:
            raise ValueError("time_step_type must be either 'dekadal' or 'monthly'")

        # Get ET image and compute ET green
        et_image = ee.Image(et_collection_list.get(i))
        et_green = compute_et_green(
            et_image, rainfed_fields, jurisdictions, et_band_name="downscaled"
        )

        et_green = back_to_int(et_green, 100)

        # Export result
        task_name = (
            f"ET_green_landsat_10m_jurisdiction_{time_step_type}_ZH_{year}-{time_step_name}"
        )
        asset_id = f"projects/thurgau-irrigation/assets/Zuerich/ET_green_landsat_10m_jurisdiction_{time_step_type}_ZH_2022/{task_name}"

        task = export_image_to_asset(et_green, asset_id, task_name, year, aoi, scale=10)
        tasks.append(task)

    return tasks

In [11]:
for year in YEARS:

    et_collection_list = ET_COLLECTION.filterDate(
        f"{year}-01-01", f"{year}-12-31"
    ).toList(TIME_STEPS)

    # DOUBLE_CROPPING_IMAGE = ee.Image(f"projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_{year}")

    tasks = process_and_export_et_green(
        et_collection_list, year, AOI, TIME_STEPS, TIME_STEP_TYPE, DOUBLE_CROPPING_IMAGE
    )

    print(f"Started {len(tasks)} export tasks.")

Exporting ET_green_landsat_10m_jurisdiction_monthly_ZH_2022-01 for 2022 to projects/thurgau-irrigation/assets/Zuerich/ET_green_landsat_10m_jurisdiction_monthly_ZH_2022/ET_green_landsat_10m_jurisdiction_monthly_ZH_2022-01
Using projection EPSG:4326 at 10m resolution
Exporting ET_green_landsat_10m_jurisdiction_monthly_ZH_2022-02 for 2022 to projects/thurgau-irrigation/assets/Zuerich/ET_green_landsat_10m_jurisdiction_monthly_ZH_2022/ET_green_landsat_10m_jurisdiction_monthly_ZH_2022-02
Using projection EPSG:4326 at 10m resolution
Exporting ET_green_landsat_10m_jurisdiction_monthly_ZH_2022-03 for 2022 to projects/thurgau-irrigation/assets/Zuerich/ET_green_landsat_10m_jurisdiction_monthly_ZH_2022/ET_green_landsat_10m_jurisdiction_monthly_ZH_2022-03
Using projection EPSG:4326 at 10m resolution
Exporting ET_green_landsat_10m_jurisdiction_monthly_ZH_2022-04 for 2022 to projects/thurgau-irrigation/assets/Zuerich/ET_green_landsat_10m_jurisdiction_monthly_ZH_2022/ET_green_landsat_10m_jurisdiction_

In [12]:
# collection.size().getInfo()

# date_utils.print_collection_dates(collection)

### Sanity check


## ET Green

In [13]:
check_collection = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Zuerich/ET_green_landsat_10m_jurisdiction_monthly_ZH_2022"
).map(lambda img: back_to_float(img, 100))

field_for_validation = ee.FeatureCollection("projects/thurgau-irrigation/assets/ZH_Nutzungsflaechen_2/2022_with_irrigation_estimates")

check_collection.first().bandNames().getInfo()

EEException: Image.bandNames: Parameter 'image' is required.

In [ ]:
check_collection_list = check_collection.toList(12)

for i in range(12):
    img = ee.Image(check_collection_list.get(i))
    print(calculate_band_std_dev(img, "ET_green").getInfo())

In [ ]:
date_utils.print_collection_dates(check_collection)
print_value_ranges(check_collection, "ET_green")

In [ ]:
Map = geemap.Map()

vis_params = {
    "bands": ["ET_green"],
    "min": 0,
    "max": 100,
    "palette": ["blue", "lightblue", "green", "yellow", "orange", "red"],
}

Map.center_object(AOI, 12)
for i in range(3, 12):
    image = ee.Image(check_collection_list.get(i))
    Map.addLayer(image, vis_params, "ET_green_" + str(i + 1))
Map.add_colorbar(vis_params, label="ET green [mm/month]")

Map

## ET Blue Post Processed

In [ ]:
et_blue_postprocessed = ee.ImageCollection("projects/thurgau-irrigation/assets/Zuerich/ET_blue_postprocessed_landsat_30m_monthly_ZH_2022").map(lambda img: back_to_float(img, 100))

In [ ]:
et_blue_postprocessed.first().bandNames().getInfo()


In [ ]:
et_blue_postprocessed_list = et_blue_postprocessed.toList(12)

In [ ]:
Map = geemap.Map()

vis_params = {
    "bands": ["ET_blue"],
    "min": 0,
    "max": 90,
    "palette": ["red", "orange", "yellow", "green"],
}

Map.center_object(AOI, 12)

for i in range(3, 10):
    image = ee.Image(et_blue_postprocessed_list.get(i))
    Map.addLayer(image, vis_params, "ET_blue_" + str(i + 1))

Map.addLayer(field_for_validation, {"color": "black"}, "Walter Koch fields")


Map.add_colorbar(vis_params, label="ET blue [mm/month]")

Map

## ET Landsat Gap Filled

In [ ]:
et_gap_filled = ee.ImageCollection("projects/thurgau-irrigation/assets/Zuerich/Landsat_ET_gap_filled_monthly_10m_ZH_2018-2022").filterDate("2022-01-01", "2022-12-31")

landsat_with_gaps = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/ETlandsatmonthly"
).filterDate("2022-01-01", "2022-12-31")
landsat_with_gaps_list = landsat_with_gaps.toList(12)

In [ ]:
et_gap_filled_list = et_gap_filled.toList(12)

In [ ]:
Map = geemap.Map()

vis_params = {
    "bands": ["ET"],
    "min": 0,
    "max": 90,
    "palette": ["red", "orange", "yellow", "green"],
}

Map.center_object(AOI, 12)

for i in range(3, 10):
    image = ee.Image(landsat_with_gaps_list.get(i))
    Map.addLayer(image, vis_params, "ET_gap" + str(i + 1))

Map.addLayer(field_for_validation, {"color": "black"}, "Walter Koch fields")

Map.add_colorbar(vis_params, label="ET gap filled [mm/month]")

Map

## Field level estimates

In [ ]:
field_level_estimates = ee.FeatureCollection("projects/thurgau-irrigation/assets/Zuerich/ET_blue_m3_ha_yr_per_field_from_monthly_landsat_30m_ZH_2022/ET_blue_m3_ha_yr_per_field_from_monthly_landsat_30m_ZH_2022")

In [ ]:
palette = ["white", "blue", "green", "yellow", "red"]

# Define visualization parameters
visualization = {
    "min": 0,
    "max": 1000,  
    "palette": palette,
}

irrigation = field_level_estimates.reduceToImage(
    properties=["ET_blue_m3_ha_yr"], reducer=ee.Reducer.first()
)

# Create a map centered on the area of interest
Map = geemap.Map(center=[47.63915833792603, 8.77542613019931], zoom=12)

# Add the gradient visualization layer to the map
Map.addLayer(irrigation, visualization, "ET_blue_m3_ha_yr")
Map.addLayer(field_for_validation, {"color": "black"}, "Walter Koch fields")



Map.add_colorbar(visualization, label="ET blue [m³/ha/yr]")

Map